<a href="https://colab.research.google.com/github/Ayesh-Sandeepa/PumpItUp/blob/main/Pump_it_up_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [271]:

import pandas as pd
import numpy as np
import sys


from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder,OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

#tr_encoder=TargetEncoder()
oh_encoder=OneHotEncoder()
lbl_encoder=LabelEncoder()


pd.set_option('display.max_columns',None)

train_values=pd.read_csv("Training set values.csv")
train_labels=pd.read_csv("Training set labels.csv")
train_labels["status_group"]=pd.Series(lbl_encoder.fit_transform(train_labels["status_group"]))

train_values["LatLong"]=train_values["longitude"]*train_values["latitude"]
train_values=train_values.drop(["latitude","longitude"], axis=1)



cats_target=["funder","ward","installer","subvillage","lga","scheme_name","construction_year","region_code","district_code","water_quality","quantity","source","waterpoint_type","source_class","extraction_type","management","payment_type"]
cats_oh=["basin","scheme_management","management_group"]
nums=["gps_height","LatLong","population"]

cats=["amount_tsh","funder","ward","installer","subvillage","lga","scheme_name","construction_year","region","region_code","district_code","basin","scheme_management","management_group","extraction_type","management","payment_type","water_quality","quantity","source","waterpoint_type","source_class","permit","public_meeting","wpt_name","region","extraction_type_group","extraction_type_class","quality_group","source_type","waterpoint_type_group"]

drop_cols=["date_recorded","num_private","recorded_by","payment","quantity_group"]

train_values[cats_target]=train_values[cats_target].fillna("missing")
train_values[cats_oh]=train_values[cats_oh].fillna("missing")

train_values[nums]=train_values[nums].fillna(train_values[nums].mean())


In [ ]:
train_values["permit"]= train_values["permit"].astype(str)
train_values["public_meeting"]= train_values["public_meeting"].astype(str)
train_permit.head()


In [ ]:
#train_values.recorded_by.unique()

#otp=pd.DataFrame(oh_encoder.fit_transform(train_values[cats_oh]))

#train_values=train_values.drop(cats_oh,axis=1)
#train_values=train_values.join(otp)

for col in cats:
    print (col)
    train_values[col]=pd.Series(lbl_encoder.fit_transform(train_values[col]))

train_values=train_values.drop(drop_cols,axis=1)

X_train, X_test, y_train, y_test = train_test_split(train_values,train_labels, test_size=0.2, random_state=25)

clf = RandomForestClassifier(n_estimators = 200, n_jobs = -1)


clf.fit(X_train, y_train["status_group"])


# Accuracy
accuracy = accuracy_score(clf.predict(X_test), y_test["status_group"])
print("Accuracy = " + str(accuracy))


In [ ]:
clf.fit(train_values, train_labels["status_group"])

In [275]:
test_values=pd.read_csv("Testing set values.csv")

test_values["LatLong"]=test_values["longitude"]*test_values["latitude"]
test_values=test_values.drop(["latitude","longitude"], axis=1)

test_values[cats_target]=test_values[cats_target].fillna("missing")
test_values[cats_oh]=test_values[cats_oh].fillna("missing")

test_values[nums]=test_values[nums].fillna(test_values[nums].mean())


test_values["permit"]=test_values["permit"].astype(str)
test_values["public_meeting"]=test_values["public_meeting"].astype(str)

In [ ]:

#train_values.recorded_by.unique()

#otp1=pd.DataFrame(oh_encoder.fit_transform(test_values[cats_oh]))

#test_values=test_values.drop(cats_oh,axis=1)
#test_values=test_values.join(otp)

for col in cats:
    print (col)
    test_values[col]=pd.Series(lbl_encoder.fit_transform(test_values[col]))

test_values=test_values.drop(drop_cols,axis=1)


In [ ]:
# Prediction for test data
prediction = clf.predict(test_values)

status_group = ["functional", "non functional", "functional needs repair"]

### Making submission file###
# Dataframe as per submission format
submission = pd.DataFrame({
            "id": test_values["id"],
            "status_group": prediction
        })
for i in range(len(status_group)):
    submission.loc[submission["status_group"] == i, "status_group"] = status_group[i]


# Store submission dataframe into file
submission.to_csv("submissionII.csv", index = False)


# New Section